CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [1]:
import numpy as np
import random
import pandas as pd

#!pip install pyomo
#import pyomo.environ as pyEnv
#!pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB
import copy
import time

adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjacency_df-n300-m1200.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2

#sets
setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
setN = adjacency_df.loc[(adjacency_df.loc[:, "IsRisky"] < 1),"IsRisky"].index.tolist()


#parameters
wij = [[adjacency_df.loc[i,j] for i in setV] for j in setV  ]
ti = [adjacency_df.loc[i,"Threshold"] for i in setV]
Xi = [adjacency_df.loc[i,"IsRisky"] for i in setV]
Blocking_Capacity = int((n) / 10) 
#Blocking_Capacity = int((n) / 10)*2

start=time.time()
#model
Math_Model = gp.Model("Math_Model")

#variables
Yi = Math_Model.addVars(setV,vtype=GRB.BINARY,name="Yi") #Bunun yerine SetS'de tanımlı olabilir!!!!
Ii = Math_Model.addVars(setV,vtype=GRB.BINARY, obj=Xi, name="Ii")
Multi= Math_Model.addVars(setV,vtype=GRB.BINARY,name="Multi") #(1-Y[i])*(1-I[i])

#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

    # Mult[i] =  (1-Y[i])*(1-I[i])   (1)
for i in setV:
    Math_Model.addConstr(Multi[i], sense=GRB.EQUAL, rhs=(1-Yi[i])*(1-Ii[i]), name="Multi constraint".format(i))#constraints
# Y[i] +I[i] <= 1   (1)
for i in setV:
    Math_Model.addConstr((Yi[i]+ Ii[i]), sense=GRB.LESS_EQUAL, rhs=1, name="Bloklanan birey infected olamaz".format(i))

#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i])*(1-I[i]) <=  t[i]    for all i in S
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*Multi[i] , sense=GRB.LESS_EQUAL, rhs=ti[i-1],name="Birinci:Tresholdu geçen birey hasta olur.".format(i))
#(Σ(over j in V  w[i][j]*I[j]))*(1-Y[i]) >  t[i]*I[i]         for all i in S 
for i in setS:
    Math_Model.addConstr(lhs=gp.quicksum(wij[i-1][j-1]*Ii[j] for j in setV)*(1-Yi[i]) , sense=GRB.GREATER_EQUAL, rhs=ti[i-1]*Ii[i],name="İkinci:Tresholdu geçen birey hasta olur.".format(i))

#Σ(over i Y[i])<= c 
Math_Model.addConstr(lhs=gp.quicksum(Yi[i] for i in setV) , sense=GRB.LESS_EQUAL, rhs=Blocking_Capacity,name="Bloklama kapasitesinden fazla bloklama yapılamaz.".format(i))

#I[i]=1       for all i in V-S(P) 
for i in setP:
    Math_Model.addConstr(Ii[i], sense=GRB.EQUAL, rhs=1, name="Başlangıçta enfekte olanların I[i] parametresi bire eşittir.".format(i))

for i in setN:
    Math_Model.addConstr(Yi[i], sense=GRB.EQUAL, rhs=0, name="Riskli olmayan bireyler bloklanamaz!".format(i))
    
    
Math_Model.write("out.lp")
# asd.computeIIS()
# asd.write("model.ilp")

Math_Model.ModelSense = GRB.MINIMIZE
Math_Model.optimize()

#for v in Math_Model.getVars():
    #if(v.x>= -0.0001):
        #print('%s %g' % (v.varName, v.x))

vars = Math_Model.getVars()

adjacency_df['Blocked']=np.zeros((n,1))

adjacency_df['Ending_Infected']=np.zeros((n,1))


for i in range(1,n+1):
    adjacency_df.at[i,'Blocked']=vars[i-1].x
    adjacency_df.at[i,'Ending_Infected']=vars[i+n-1].x

Objective_Value = Math_Model.ObjVal

adjacency_df.loc[(adjacency_df.loc[:,'IsRisky'] == 1) & (adjacency_df.loc[:,'Ending_Infected'] == 1) ,['Threshold','IsInfected','IsRisky','Blocked','Ending_Infected']]

adjacency_df['Blocked'].to_csv("Math_Model_NODE-n"+str(n)+'-m'+str(m)+'-OBJ'+str(Objective_Value)+".csv",index=False) 
end=time.time()

Academic license - for non-commercial use only - expires 2021-05-20
Using license file C:\Users\Lenovo\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 841 rows, 900 columns and 1740 nonzeros
Model fingerprint: 0x6d602fd5
Model has 840 quadratic constraints
Variable types: 0 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-03, 1e+00]
  QLMatrix range   [1e-03, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
  QRHS range       [2e-03, 1e+00]
Presolve removed 841 rows and 900 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 55 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.500000000000e+01

In [2]:
print(end-start)

5.3209521770477295


In [3]:
ati=adjacency_df['Blocked'].tolist()

In [4]:
adjacency_df.loc[adjacency_df['Ending_Infected']==1,'IsRisky'].sum()

55.0

In [5]:
adjacency_df.loc[adjacency_df['Blocked']==1,'IsRisky'].sum()

30.0

In [6]:
adjacency_df.loc[adjacency_df['Blocked']==1,'IsRisky']

26     1.0
28     1.0
29     1.0
38     1.0
47     1.0
49     1.0
55     1.0
57     1.0
61     1.0
65     1.0
107    1.0
127    1.0
136    1.0
149    1.0
158    1.0
170    1.0
183    1.0
192    1.0
194    1.0
203    1.0
211    1.0
216    1.0
218    1.0
243    1.0
257    1.0
266    1.0
269    1.0
271    1.0
277    1.0
281    1.0
Name: IsRisky, dtype: float64